 Paso 1.
# Realice la limpieza de los datasets:
# - Renombre la columna "default payment next month" a "default".
# - Remueva la columna "ID".
# - Elimine los registros con informacion no disponible.
# - Para la columna EDUCATION, valores > 4 indican niveles superiores re
#   de educación, agrupe estos valores en la categoría "others".
# - Renombre la columna "default payment next month" a "default"
# - Remueva la columna "ID".

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score,
    recall_score, f1_score, confusion_matrix, classification_report
)
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,precision_score, recall_score,ConfusionMatrixDisplay
from scipy.stats import randint
import pandas as pd

    # Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import os
import json

In [3]:
def load_data(name):
    dataframe = pd.read_csv(
        f"../files/input/{name}",
        index_col=False,
        compression="zip",
    )

    

    return dataframe

In [4]:
#nombre de archivos
name_test="test_data.csv.zip"
name_train="train_data.csv.zip"

In [5]:
#carga de datos train
df_train=load_data(name_train)

In [6]:
#carga de datos test
df_test=load_data(name_test)

In [7]:

df_train.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,10748,310000,1,3,1,32,0,0,0,0,...,84373,57779,14163,8295,6000,4000,3000,1000,2000,0
1,12574,10000,2,3,1,49,-1,-1,-2,-1,...,1690,1138,930,0,0,2828,0,182,0,1
2,29677,50000,1,2,1,28,-1,-1,-1,0,...,45975,1300,43987,0,46257,2200,1300,43987,1386,0
3,8857,80000,2,3,1,52,2,2,3,3,...,40748,39816,40607,3700,1600,1600,0,1600,1600,1
4,21099,270000,1,1,2,34,1,2,0,0,...,22448,15490,17343,0,4000,2000,0,2000,2000,0


In [8]:
df_test.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
1,10,20000,1,3,2,35,-2,-2,-2,-2,...,0,13007,13912,0,0,0,13007,1122,0,0
2,11,200000,2,3,2,34,0,0,2,0,...,2513,1828,3731,2306,12,50,300,3738,66,0
3,15,250000,1,1,2,29,0,0,0,0,...,59696,56875,55512,3000,3000,3000,3000,3000,3000,0
4,16,50000,2,3,3,23,1,2,0,0,...,28771,29531,30211,0,1500,1100,1200,1300,1100,0


In [9]:
print(len(df_train))
print(len(df_test))

21000
9000


In [10]:
df_train['EDUCATION']=df_train['EDUCATION'].replace(0,np.nan)
df_train['MARRIAGE']=df_train['MARRIAGE'].replace(0,np.nan)

df_test['EDUCATION']=df_test['EDUCATION'].replace(0,np.nan)
df_test['MARRIAGE']=df_test['MARRIAGE'].replace(0,np.nan)

df_train.dropna(inplace=True)
df_test.dropna(inplace=True)

In [11]:
df_train['MARRIAGE'].unique()

array([1., 2., 3.])

In [12]:
print(len(df_train))
print(len(df_test))

20953
8979


In [13]:
#renombrar Columnas
df_train.rename(columns={"default payment next month":"default"},inplace=True)

In [14]:
#renombrar Columnas test
df_test.rename(columns={"default payment next month":"default"},inplace=True)

In [15]:
#quitar ID 
df_train=df_train.drop(columns="ID")
df_test=df_test.drop(columns="ID")


In [16]:
df_train.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,310000,1,3.0,1.0,32,0,0,0,0,0,...,84373,57779,14163,8295,6000,4000,3000,1000,2000,0
1,10000,2,3.0,1.0,49,-1,-1,-2,-1,2,...,1690,1138,930,0,0,2828,0,182,0,1
2,50000,1,2.0,1.0,28,-1,-1,-1,0,-1,...,45975,1300,43987,0,46257,2200,1300,43987,1386,0
3,80000,2,3.0,1.0,52,2,2,3,3,3,...,40748,39816,40607,3700,1600,1600,0,1600,1600,1
4,270000,1,1.0,2.0,34,1,2,0,0,2,...,22448,15490,17343,0,4000,2000,0,2000,2000,0


In [17]:
filterValue=df_train[df_train["default"]==0]
filterValue

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,310000,1,3.0,1.0,32,0,0,0,0,0,...,84373,57779,14163,8295,6000,4000,3000,1000,2000,0
2,50000,1,2.0,1.0,28,-1,-1,-1,0,-1,...,45975,1300,43987,0,46257,2200,1300,43987,1386,0
4,270000,1,1.0,2.0,34,1,2,0,0,2,...,22448,15490,17343,0,4000,2000,0,2000,2000,0
5,140000,2,3.0,1.0,30,0,0,0,0,2,...,82227,65000,60848,4700,4505,12906,0,2210,2300,0
6,200000,1,2.0,2.0,26,-1,-1,0,0,0,...,16017,12613,6600,12957,3884,5010,700,360,1713,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20995,140000,2,2.0,1.0,27,2,-1,-1,-1,0,...,1580,804,728,752,800,1580,0,700,700,0
20996,130000,1,2.0,2.0,41,0,0,0,0,0,...,123107,42897,39378,4442,5200,5012,2500,5000,2000,0
20997,50000,1,3.0,2.0,23,0,0,0,0,0,...,28967,29829,30046,1973,1426,1001,1432,1062,997,0
20998,90000,2,3.0,2.0,25,0,0,0,0,0,...,5613,10113,10113,3000,3000,0,4500,0,3440,0


In [18]:
filterValue=df_train[df_train["default"]==1]
filterValue

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
1,10000,2,3.0,1.0,49,-1,-1,-2,-1,2,...,1690,1138,930,0,0,2828,0,182,0,1
3,80000,2,3.0,1.0,52,2,2,3,3,3,...,40748,39816,40607,3700,1600,1600,0,1600,1600,1
11,20000,1,3.0,2.0,23,0,0,0,0,0,...,12307,12703,12970,1161,2701,1204,600,620,622,1
16,140000,2,3.0,2.0,26,2,0,0,0,0,...,140103,142750,142855,5301,5400,5100,5300,5500,5400,1
33,130000,1,1.0,2.0,46,0,0,0,0,0,...,40177,41529,43734,6000,9000,2000,2000,3000,4000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20975,150000,1,1.0,1.0,56,2,0,0,0,0,...,40813,42164,43465,5006,3000,1452,5000,5000,2000,1
20983,140000,2,2.0,1.0,36,-1,-1,-1,-1,0,...,1031,487,863,341,165,1031,0,863,487,1
20986,200000,2,2.0,1.0,45,0,0,0,0,0,...,121888,66771,65495,4543,5000,4500,3100,3000,1300,1
20987,210000,1,1.0,2.0,27,1,-1,-1,-1,-1,...,582,291,1027,582,0,582,291,1027,291,1


In [19]:
df_train.dtypes

LIMIT_BAL      int64
SEX            int64
EDUCATION    float64
MARRIAGE     float64
AGE            int64
PAY_0          int64
PAY_2          int64
PAY_3          int64
PAY_4          int64
PAY_5          int64
PAY_6          int64
BILL_AMT1      int64
BILL_AMT2      int64
BILL_AMT3      int64
BILL_AMT4      int64
BILL_AMT5      int64
BILL_AMT6      int64
PAY_AMT1       int64
PAY_AMT2       int64
PAY_AMT3       int64
PAY_AMT4       int64
PAY_AMT5       int64
PAY_AMT6       int64
default        int64
dtype: object

In [20]:
unique_values_col=df_train["EDUCATION"].unique()
unique_values_col

array([3., 2., 1., 5., 4., 6.])

In [21]:
# añadir la categoria others con los valores de eduación mayores a 4 =1 y menores y menores o iguales con valor 0
df_train.loc[df_train["EDUCATION"]>4,"EDUCATION"]=4
df_test["EDUCATION"]=df_test["EDUCATION"].apply(lambda x: 4 if x>4 else x)


In [22]:
df_train.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default
0,310000,1,3.0,1.0,32,0,0,0,0,0,...,84373,57779,14163,8295,6000,4000,3000,1000,2000,0
1,10000,2,3.0,1.0,49,-1,-1,-2,-1,2,...,1690,1138,930,0,0,2828,0,182,0,1
2,50000,1,2.0,1.0,28,-1,-1,-1,0,-1,...,45975,1300,43987,0,46257,2200,1300,43987,1386,0
3,80000,2,3.0,1.0,52,2,2,3,3,3,...,40748,39816,40607,3700,1600,1600,0,1600,1600,1
4,270000,1,1.0,2.0,34,1,2,0,0,2,...,22448,15490,17343,0,4000,2000,0,2000,2000,0


In [23]:
unique_values_col=df_train["EDUCATION"].unique()
unique_values_col


array([3., 2., 1., 4.])

In [24]:
filterValue=df_train[df_train["EDUCATION"]==6]
filterValue

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default


Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.


In [25]:
#Dividir el dataframe
from sklearn.model_selection import train_test_split
x_train=df_train.drop("default",axis=1)
y_train=df_train["default"]

x_test=df_test.drop("default",axis=1)
y_test=df_test["default"]



In [26]:
unique_values_col=df_test["EDUCATION"].unique()
unique_values_col

array([2., 3., 1., 4.])

In [27]:
print(len(x_train),len(y_train),len(x_test),len(y_test))

20953 20953 8979 8979


In [28]:
x_train.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'],
      dtype='object')

Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Ajusta un modelo de bosques aleatorios (rando forest).

In [29]:
categorical_features=['SEX', 'EDUCATION', 'MARRIAGE']
numerical_features= ['LIMIT_BAL', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT2',
       'BILL_AMT3', 'BILL_AMT4', 'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1',
       'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6' ]



In [30]:
# Preprocesamiento
preprocessor = ColumnTransformer(
    transformers = [
        ("encoder", OneHotEncoder(), categorical_features),("scaler",StandardScaler(),numerical_features)
    ],
    remainder = "passthrough"
)

In [31]:
# # Pipeline
# pipeline = Pipeline([
#     ("preprocessor", preprocessor),
#     ("select_kbest", SelectKBest(score_func=f_classif, k=5)),
#     ("pca", PCA(n_components=5)),
#     ("RandomForest", RandomForestClassifier())
# ])

In [32]:
# Pipeline
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("RandomForest", RandomForestClassifier())
])

Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use la función de precision
# balanceada para medir la precisión del modelo.

In [33]:
# param_grid = {
#     'RandomForest__n_estimators': [100,200,500],
#     'RandomForest__max_depth': [5,10,15],
#     "pca__n_components": [5,10,20],
#     "select_kbest__k": [5,10,24],
# }

In [34]:
param_grid = {
    'RandomForest__n_estimators': [200],
    'RandomForest__max_depth': [20],
    'RandomForest__criterion':['gini'],
    'RandomForest__random_state':[42500]
}

In [35]:
grid_search = GridSearchCV(
    pipeline, param_grid, cv=10, scoring="balanced_accuracy", verbose=2, n_jobs=-1 
)

In [36]:
grid_search.fit(x_train, y_train)



Fitting 10 folds for each of 1 candidates, totalling 10 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('encoder',
                                                                         OneHotEncoder(),
                                                                         ['SEX',
                                                                          'EDUCATION',
                                                                          'MARRIAGE']),
                                                                        ('scaler',
                                                                         StandardScaler(),
                                                                         ['LIMIT_BAL',
                                                                          'AGE',
                                                                          'PAY_0',
                                                                          'PAY_2',
                                                                          'PAY_3',
                                                                          'PAY_4',
                                                                          'PAY_5',
                                                                          'PAY_6',
                                                                          'BILL_AMT1',
                                                                          'BILL_AMT2',
                                                                          'BILL_AMT3',
                                                                          'BILL_AMT4',
                                                                          'BILL_AMT5',
                                                                          'BILL_AMT6',
                                                                          'PAY_AMT1',
                                                                          'PAY_AMT2',
                                                                          'PAY_AMT3',
                                                                          'PAY_AMT4',
                                                                          'PAY_AMT5',
                                                                          'PAY_AMT6'])])),
                                       ('RandomForest',
                                        RandomForestClassifier())]),
             n_jobs=-1,
             param_grid={'RandomForest__criterion': ['gini'],
                         'RandomForest__max_depth': [20],
                         'RandomForest__n_estimators': [200],
                         'RandomForest__random_state': [42500]},
             scoring='balanced_accuracy', verbose=2)

In [37]:
# Mejor modelo
print("Mejores hiperparámetros:", grid_search.best_params_)
best_model = grid_search

Mejores hiperparámetros: {'RandomForest__criterion': 'gini', 'RandomForest__max_depth': 20, 'RandomForest__n_estimators': 200, 'RandomForest__random_state': 42500}


Paso 5.
# Guarde el modelo como "files/models/model.pkl".

In [38]:
import gzip

In [39]:
def save_estimator(best_model):

    import pickle
    

    with gzip.open("../files/models/model.pkl.gz", "wb") as file:
        pickle.dump(best_model, file) 

    

In [40]:
save_estimator(best_model)

 Paso 6.
# Calcule las metricas de precision, precision balanceada, recall,
# y f1-score para los conjuntos de entrenamiento y prueba.
# Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# Este diccionario tiene un campo para indicar si es el conjunto
# de entrenamiento o prueba. Por ejemplo:
#
# {'dataset': 'train', 'precision': 0.8, 'balanced_accuracy': 0.7, 'recall': 0.9, 'f1_score': 0.85}
# {'dataset': 'test', 'precision': 0.7, 'balanced_accuracy': 0.6, 'recall': 0.8, 'f1_score': 0.75}

Paso 7.
# Calcule las matrices de confusion para los conjuntos de entrenamiento y
# prueba. Guardelas en el archivo files/output/metrics.json. Cada fila
# del archivo es un diccionario con las metricas de un modelo.
# de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'cm_matrix', 'dataset': 'train', 'true_0': {"predicted_0": 15562, "predicte_1": 666}, 'true_1': {"predicted_0": 3333, "predicted_1": 1444}}
# {'type': 'cm_matrix', 'dataset': 'test', 'true_0': {"predicted_0": 15562, "predicte_1": 650}, 'true_1': {"predicted_0": 2490, "predicted_1": 1420}}
#

In [41]:
# Predicciones
y_train_pred = best_model.predict(x_train)
y_test_pred = best_model.predict(x_test)

In [42]:
def load_estimator():

    import os
    import pickle

    if not os.path.exists("../files/models/model.pkl"):
        return None
    with open("../files/models/model.pkl", "rb") as file:
        estimator = pickle.load(file)

    return estimator

In [43]:
cm_train=confusion_matrix(y_train, y_train_pred)
cm_test=confusion_matrix(y_test, y_test_pred)

In [51]:
# Métricas
metrics_train={
        "type":"metrics",
        "dataset":"train",
        "precision":precision_score(y_train, y_train_pred),
        "balanced_accuracy": balanced_accuracy_score(y_train, y_train_pred),
        "recall": recall_score(y_train, y_train_pred),
        "f1_score": f1_score(y_train, y_train_pred)
    }

metrics_test={
        "type":"metrics",
        "dataset":"test",
        "precision": precision_score(y_test, y_test_pred),
        "balanced_accuracy": balanced_accuracy_score(y_test, y_test_pred),
        "recall": recall_score(y_test, y_test_pred),
        "f1_score": f1_score(y_test, y_test_pred)
    }
cm_matrix_train={
    "type": "cm_matrix",
    "dataset": "train",
    "true_0": {
        "predicted_0": int(cm_train[0][0]),  # Verdaderos Negativos
        "predicted_1": int(cm_train[0][1])   # Falsos Positivos
    },
    "true_1": {
        "predicted_0": int(cm_train[1][0]),  # Falsos Negativos
        "predicted_1": int(cm_train[1][1])   # Verdaderos Positivos
    }
}
cm_matrix_test={
    "type": "cm_matrix",
    "dataset": "test",
    "true_0": {
        "predicted_0": int(cm_test[0][0]),  # Verdaderos Negativos
        "predicted_1": int(cm_test[0][1])   # Falsos Positivos
    },
    "true_1": {
        "predicted_0": int(cm_test[1][0]),  # Falsos Negativos
        "predicted_1": int(cm_test[1][1])   # Verdaderos Positivos
    }

}
    





In [52]:
metrics=[metrics_train,metrics_test,cm_matrix_train,cm_matrix_test]
pd.DataFrame(metrics).to_json('../files/output/metrics.json',orient='records',lines=True)

In [46]:
#  # Guardar las métricas en un archivo JSON
# output_file_path = '../files/output/metrics.json'
# os.makedirs(os.path.dirname(output_file_path), exist_ok=True)  # Crear directorios si no existen

# with open(output_file_path, 'w') as json_file:
#     json.dump(metrics, json_file, indent=4)


In [47]:
# # Guardar las métricas en un archivo JSON
# output_file_path = '../files/output/metrics.json'
# os.makedirs(os.path.dirname(output_file_path), exist_ok=True)  # Crear directorios si no existen
# with open(output_file_path, 'w',encoding='utf-8') as json_file:
#     # for metric in metrics:
#     json.dump(metrics, json_file, indent=4)
#         # json_file.write('\n')

In [48]:
def _load_metrics():
    metrics = []
    with open("../files/output/metrics.json", "r", encoding="utf-8") as file:
        for line in file:
            metrics.append(json.loads(line))
    return metrics

In [49]:
_load_metrics()

[{'type': 'metrics',
  'dataset': 'train',
  'precision': 1.0,
  'balanced_Accuracy': 0.8936507937,
  'recall': 0.7873015873,
  'f1-Score': 0.8809946714,
  'Balanced_Accuracy': None,
  'Recall': None,
  'F1-Score': None,
  'true_0': None,
  'true_1': None},
 {'type': 'metrics',
  'dataset': 'test',
  'precision': 0.6635111876,
  'balanced_Accuracy': None,
  'recall': None,
  'f1-Score': None,
  'Balanced_Accuracy': 0.6746157112,
  'Recall': 0.4045120672,
  'F1-Score': 0.5026075619,
  'true_0': None,
  'true_1': None},
 {'type': 'cm_matrix',
  'dataset': 'train',
  'precision': None,
  'balanced_Accuracy': None,
  'recall': None,
  'f1-Score': None,
  'Balanced_Accuracy': None,
  'Recall': None,
  'F1-Score': None,
  'true_0': {'predicted_0': 16228, 'predicted_1': 0},
  'true_1': {'predicted_0': 1005, 'predicted_1': 3720}},
 {'type': 'cm_matrix',
  'dataset': 'test',
  'precision': None,
  'balanced_Accuracy': None,
  'recall': None,
  'f1-Score': None,
  'Balanced_Accuracy': None,
  'R